In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import BytesIO
import unidecode

In [2]:
pagina_relatorios = r'https://www.prefeitura.sp.gov.br/cidade/secretarias/licenciamento/servicos/index.php?p=3334'

In [3]:
def pegar_links_relatorios(pagina_relatorios):
    
    
    with requests.get(pagina_relatorios) as r:
        html = r.text
    sopa = BeautifulSoup(html)
    tabelas = sopa.find_all('tbody')
    links = []
    for table in tabelas:
        links_header = table.find_all('th')
        links.extend(links_header)
        links_rows = table.find_all('tr')
        links.extend(links_rows)
    links = [link.find('a')['href'] for link in links]
    links = [link for link in links if link.endswith('.xls') or link.endswith('.xlsx')]
    return links

In [4]:
links = pegar_links_relatorios(pagina_relatorios)
anuais = {int(''.join([char for char in link.split('/')[-1] if char.isdigit()][:4])) : link
              for link in links if "anual" in link.lower()}

In [5]:
def baixar_planilha(link, salvar = False):
    
    with requests.get(link) as r:
        excel = r.content
    file_handle = BytesIO(excel)
    df  = pd.read_excel(file_handle)
    if salvar:
        filename = link.split('/')[-1]
        pd.to_excel(filename)
    return df

In [6]:
def rename_cols(df):
    
    cols_df = df.columns
    cols_novas = [unidecode.unidecode(col.lower().strip())
                 for col in cols_df]
    mapper = dict(zip(cols_df, cols_novas))
    
    df.rename(mapper, axis =1, inplace=  True)
    

In [7]:
def juntar_tudo_anual(anuais, salvar = False):
    
    dfs = []
    for ano, link in anuais.items():
        if ano > 2015: #antes disso tem que arrumar muito os dados
            df = baixar_planilha(link, salvar)
            df['ANO'] = ano
            rename_cols(df)
            dfs.append(df)
    final = pd.concat(dfs, sort = False)
    
    return final

In [37]:
def gerar_tempo_aprov(df):
    
    df['aprovacao_dt'] = pd.to_datetime(df['aprovacao'], format = '%d/%m/%Y')
    df['data_autuacao_dt'] = pd.to_datetime(df['data autuacao'],  format = '%d/%m/%Y')
    df['tempo_aprov'] = df['aprovacao_dt'] - df['data_autuacao_dt']

In [38]:
tudo = juntar_tudo_anual(anuais)
gerar_tempo_aprov(tudo)

In [39]:
tudo['tempo_aprov']

0       761 days
1      3095 days
2      2227 days
3      2227 days
4       646 days
          ...   
5523    184 days
5524    171 days
5525   1540 days
5526   1127 days
5527   3655 days
Name: tempo_aprov, Length: 21477, dtype: timedelta64[ns]

In [40]:
for desc in tudo['descricao'].unique():
    if 'CONCLUSAO' in desc:
        print(desc)

APOSTILAMENTO DO CERTIFICADO DE CONCLUSAO
RECONSIDERACAO DE DESPACHO DE CERTIFICADO DE CONCLUSAO
CERTIFICADO DE CONCLUSAO


In [41]:
tudo['habite_se'] = tudo['descricao'].apply(lambda x: 'CONCLUSAO' in x)

In [42]:
habite_se = tudo[tudo['habite_se']].copy()

In [43]:
habite_se['count'] = 1

In [44]:
habite_se['mes_aprov'] = habite_se['aprovacao_dt'].apply(lambda x: x.month)

In [45]:
habite_se['ano_aprov'] = habite_se['aprovacao_dt'].apply(lambda x: x.year)

In [55]:
habite_se['media_tempo_aprov_dias'] = habite_se['tempo_aprov'].apply(lambda x: int(x.days))

In [58]:
habite_se.groupby(['ano_aprov', 'mes_aprov']).mean()[['media_tempo_aprov_dias']]

media_tempo_aprov_dias
ano_aprov mes_aprov                        
2016      1                       30.500000
          2                       22.500000
          3                       19.000000
          4                       86.500000
          5                       21.000000
          6                       18.000000
          7                       54.666667
          9                       34.000000
          10                      78.000000
          11                      21.000000
          12                      47.500000
2017      1                       19.000000
          2                       48.000000
          3                       13.000000
          4                       66.250000
          5                      123.500000
          6                       52.000000
          7                       55.500000
          8                       72.000000
          9                      343.000000
          10                      20.333333
          11                      62.500000
          12                     115.333333
2018      1                      185.000000
          2                        4.000000
          5                       22.000000
          6                       40.000000
          7                       39.750000
          8                       36.500000
          9                       27.500000
          10                      14.400000
          11                      34.000000
          12                       9.000000
2019      1                       16.571429
          2                       20.666667
          3                       50.000000
          4                       14.750000
          5                       21.000000
          6                       40.333333
          7                       24.600000
          8                       41.833333
          9                       39.000000
          10                     110.000000
          11                      42.666667
          12                      43.500000
2020      1                      117.111111
          3                       42.333333
          4                       40.500000
          5                       59.500000